# Database Setup

- Das Notebook erstellt die Datenbank und beläd diese mit den Demo Daten

## Alle nötigen Libraries installieren und importieren

In [4]:
!pip install soda-core==3.3.5
!pip install soda-core-postgres==3.3.5
!pip install pandas==2.2.2
!pip install psycopg2-binary==2.9.9
!pip install requests==2.32.3
!pip install matplotlib==3.9.1


[notice] A new release of pip is available: 23.2.1 -> 24.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.2.1 -> 24.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.2.1 -> 24.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.2.1 -> 24.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.2.1 -> 24.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.2.1 -> 24.2
[notice] To update, run: pip install --upgrade pip


In [5]:
import pandas as pd
import psycopg2
from psycopg2 import sql
import requests
from io import StringIO

## Start des Postgres Server (siehe auch READM.md)

- Führe PostgreSQL Server mit Docker aus:
  ```
  sudo docker run --name postgresql1 -e POSTGRES_PASSWORD=test -v ${HOME}/postgres-data/:/var/lib/postgresql/data -p 5432:5432 -d postgres:15.2
  ```


# Dowload der Demo Daten

In [6]:
CSV_URL = "https://raw.githubusercontent.com/sodadata/tutorial-demo-project/main/data/ny_bus_breakdown/bus_breakdown_and_delays.csv"

# Download the CSV file
response = requests.get(CSV_URL)
if response.status_code != 200:
    raise Exception(f"Failed to download CSV file from {CSV_URL}")

# Read the CSV file into a pandas DataFrame
csv_data = StringIO(response.text)
df = pd.read_csv(csv_data)
df

,school_year,busbreakdown_id,run_type,bus_no,route_number,reason,schools_serviced,occured_on,created_on,boro,...,how_long_delayed,number_of_students_on_the_bus,has_contractor_notified_schools,has_contractor_notified_parents,have_you_alerted_opt,informed_on,incident_number,last_updated_on,breakdown_or_running_late,school_age_or_prek
0,2017-2018,1363818,Special Ed AM Run,19250,X414,Heavy Traffic,12486,2017-08-01 07:15:00,2017-08-01 07:22:00,Bronx,...,25-30 MIN,0,Yes,Yes,Yes,2017-08-01 07:22:00,NaN,2017-08-01 07:22:55,Running Late,School-Age
1,2017-2018,1363819,Pre-K/EI,12495,B0203A,Other,C244,2017-08-01 07:21:00,2017-08-01 07:24:00,Brooklyn,...,15 MINS,0,Yes,Yes,Yes,2017-08-01 07:24:00,NaN,2017-08-01 09:42:39,Running Late,Pre-K
2,2017-2018,1363820,Special Ed AM Run,31338,X105,Heavy Traffic,12084,2017-08-01 07:20:00,2017-08-01 07:26:00,Bronx,...,25-30 MIN,3,Yes,Yes,Yes,2017-08-01 07:26:00,NaN,2017-08-01 07:26:38,Running Late,School-Age
3,2017-2018,1363821,Pre-K/EI,1050,GRA1,Other,C956,2017-08-01 07:24:00,2017-08-01 07:27:00,Queens,...,15-20MINS,8,Yes,Yes,Yes,2017-08-01 07:27:00,NaN,2017-08-01 07:28:04,Running Late,Pre-K
4,2017-2018,1363822,Pre-K/EI,2604,NYM-2,Heavy Traffic,E203,2017-08-01 07:31:00,2017-08-01 07:28:00,Bronx,...,20 MINS,5,Yes,Yes,No,2017-08-01 07:28:00,NaN,2017-08-01 07:28:17,Running Late,Pre-K
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199993,2017-2018,1368830,Special Ed AM Run,5791D,Q690,Heavy Traffic,30502,2017-09-19 07:57:00,2017-09-19 07:58:00,Queens,...,16-30 Min,12,Yes,Yes,No,2017-09-19 07:58:00,NaN,1900-01-01 00:00:00,Running Late,School-Age
199994,2015-2016,1213955,Pre-K/EI,9309,1,Heavy Traffic,S188,2015-09-10 08:10:00,2015-09-10 08:19:00,Bronx,...,15 minutes,7,Yes,No,No,2015-09-10 08:19:00,NaN,2015-09-10 08:19:28,Running Late,Pre-K
199995,2015-2016,1218292,Pre-K/EI,730,3,Mechanical Problem,C037,2015-09-30 13:43:00,2015-09-30 13:47:00,Bronx,...,25,0,Yes,Yes,Yes,2015-09-30 13:47:00,NaN,2015-09-30 13:47:38,Running Late,Pre-K
199996,2015-2016,1225246,Special Ed AM Run,1309,J737,Heavy Traffic,75003,2015-10-27 07:51:00,2015-10-27 08:08:00,Manhattan,...,20,0,Yes,Yes,Yes,2015-10-27 08:08:00,NaN,2015-10-27 08:32:23,Running Late,School-Age


# Anlegen der Testdatenbank und Tabelle

In [7]:
host = 'localhost'
database = 'demo'
user = 'postgres'
password = 'test'
schema = 'public'
table_name = 'bus_breakdown_and_delays'

def connect(database='demo'):
    """Connect to database"""
    conn = None
    try:
        print('Connecting...')
        conn = psycopg2.connect(
            host=host,
            database=database,
            user=user,
            password=password)
    except (Exception, psycopg2.DatabaseError) as error:
        #print(error)
        # Connect to the default database (usually 'postgres') to create the new database
        conn = psycopg2.connect(
            host=host,
            database="postgres",
            user=user,
            password=password,
        )
        cur = conn.cursor()
        conn.autocommit = True
        # Create the new database
        try:
            cur.execute(sql.SQL("CREATE DATABASE {}").format(sql.Identifier(database)))
            print(f"Database '{database}' created successfully")
        except (Exception, psycopg2.DatabaseError) as e:
            print(f"Failed to create database '{database}': {e}")
        cur.close()
        conn.close()
        return connect()

    print(f'All good, connection to {database} successful!')
    return conn


## Schema anlegen

In [8]:
# Define the table creation SQL
create_table_sql = """
CREATE TABLE IF NOT EXISTS bus_breakdown_and_delays (
    school_year VARCHAR(50),
    busbreakdown_id INTEGER PRIMARY KEY,
    run_type TEXT,
    bus_no TEXT,
    route_number TEXT,
    reason TEXT,
    schools_serviced TEXT,
    occurred_on TIMESTAMP,
    created_on TIMESTAMP,
    boro TEXT,
    bus_company_name TEXT,
    how_long_delayed TEXT,
    number_of_students_on_the_bus INTEGER,
    has_contractor_notified_schools TEXT,
    has_contractor_notified_parents TEXT,
    have_you_alerted_opt TEXT,
    informed_on TIMESTAMP,
    incident_number TEXT,
    last_updated_on TIMESTAMP,
    breakdown_or_running_late TEXT,
    school_age_or_prek TEXT
)
"""
conn = connect()
cur = conn.cursor()
try:
    cur.execute(create_table_sql)
    print(f"Table '{table_name}' created successfully or already exists")
    conn.commit()
except Exception as e:
    print(f"Failed to create table '{table_name}': {e}")
finally:
    cur.close()
    conn.close()


Connecting...
Database 'demo' created successfully
Connecting...
All good, connection to demo successful!
Table 'bus_breakdown_and_delays' created successfully or already exists


## Daten importieren

In [9]:
conn = connect()
cur = conn.cursor()

# Create a temporary CSV file in memory without the header
csv_buffer = StringIO()
df.to_csv(csv_buffer, index=False, header=False)
csv_buffer.seek(0)

# Copy the DataFrame to the PostgreSQL table
try:
    cur.copy_expert(f"COPY {table_name} FROM STDIN WITH CSV", csv_buffer)
    conn.commit()
    print("CSV file imported successfully into PostgreSQL")
except Exception as e:
    print(e)
    conn.rollback()
    print(f"Failed to import CSV file into PostgreSQL: {e}")
finally:
    cur.close()
    conn.close()


Connecting...
All good, connection to demo successful!
CSV file imported successfully into PostgreSQL
